In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [9]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_co_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [10]:
wiki.name

'wikipedia'

In [4]:
from langchain_community.document_loaders import WebBaseLoader # web based loader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS #FAISS Vector database
from langchain_community.embeddings import OllamaEmbeddings

# load, chunk, and index the content of the HTML page
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OllamaEmbeddings())
retriver=vectordb.as_retriever()
retriver

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000182754F2CE0>)

In [5]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriver,"langsmith_search",
                      "Search for information about LangSmith. For any questions about langsmith you must use this tool."
)

In [6]:
retriever_tool.name

'langsmith_search'

In [8]:
##Arxiv tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [12]:
tools=[wiki,arxiv,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from "e:\\GitHub repo's\\Learnings-and-Revisions\\Artificial Intelligence\\GenAI\\LangChain\\Krish\\April2024\\ChatbOt\\lib\\site-packages\\wikipedia\\__init__.py">, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=4000)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200, arxiv_result=<class 'arxiv.Result'>)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about langsmith you must use this tool.', args_schema=<class 'langchain.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001827D58E680>, retriever=Vector

In [14]:
##Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

NameError: name 'llm' is not defined

In [ ]:
load_dotenv()
import os 
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model="gpt-3.5-turbo-0125",temperature=0)

In [13]:
from langchain import hub
#Get the prompt to use - you can modify this!
prompt=hub.pull("hwchase17/openai-functions-agents")
prompt.messages

ImportError: Could not import langchainhub, please install with `pip install langchainhub`.

In [ ]:
##Agent executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

In [ ]:
agent_executor.invoke({"input":"Tell me about langsmith"}